# 1. Eindige velden

In [1]:
from FiniteField import *
from util import *

## 1.1 Rekenen modulo $p$

We experimenteren eerst binnen het veld $\mathbb{Z}_5$

In [2]:
Z5 = IntegerField(5)
print("Z5 =", Z5)

Z5 = [0, 1, 2, 3, 4]


We kunnen gewoon door dit veld itereren alsof het een lijst zou zijn:

In [3]:
zero, one, two, three, four = tuple([elem for elem in Z5])
Z5.zero(), Z5.one(), Z5[4]

(0, 1, 4)

Let op dat dit geen getallen zijn, maar restklassen modulo 5. Je kunt er wel mee (modulo-)rekenen alsof het getallen waren.

In [4]:
one + two

3

In [5]:
three * four

2

In [6]:
two / three

4

Ook veelvouden en machten:

In [7]:
2 * three

1

In [8]:
three ** 2

4

Vermenigvuldiging en deling worden efficient uitgevoerd omdat we een generator hebben.

In [9]:
Z5.generator()

2

In [10]:
for i in range(len(Z5)):
    print(i, "→", Z5.generator_to_power(i))

0 → 1
1 → 2
2 → 4
3 → 3
4 → 1


Of omgekeerd: de exponent opzoeken

In [11]:
for elem in Z5:
    if not elem.is_zero():
        print(elem, "←", Z5.generator_exponent(elem))

1 ← 0
2 ← 1
3 ← 3
4 ← 2


## 1.2 Veeltermen

We kunnen ook veeltermen definieren

In [12]:
Polynomial([two, three, zero, one])

2 + 3x + x³

En die hebben hun eigen algebra:

In [13]:
p = Polynomial.one(Z5) + Polynomial.x_to_power(2, Z5)
q = Polynomial([two, two])
print(p)
print(q)
print(p.multiply_x_to_power(5))
print(two * q)
print(p * q)
print(p // q) # deling waarbij de rest genegeerd wordt
print(p % q)

1 + x²
2 + 2x
x⁵ + x⁷
4 + 4x
2 + 2x + 2x² + 2x³
2 + 3x
2


Gehele deling is altijd gedefinieerd, maar let erop dat er een verschil is tussen "/" en "//". Niet alle veeltermen kunnen met "/" door elkaar gedeeld worden.

In [14]:
try:
    p / q
except ValueError as e:
    print(e)

Cannot divide these two polynomials. The remainder is non-zero.


Toepassen op een element is zoals je zou verwachten:

In [15]:
p(one)

2

Nulpunten en factorisatie:

In [16]:
for root in p.find_roots():
    print(root, end=", ")

2, 3, 

In [17]:
factors = p.factor()
print(factors)
print(product(factors) == p)

[2 + x, 3 + x]
True


In [18]:
p.is_reducible()

True

Wat informatie over de coefficienten:

In [19]:
for coef in p:
    print(coef, end=", ")

1, 0, 1, 

In [20]:
for i in range(len(p)):
    print(p[i], end=", ")

1, 0, 1, 

In [21]:
len(p), p.degree()

(3, 2)

## 1.3 Uitbreidingsvelden

We zoeken een primitieve veelterm $w(x)$ van graad 2 in $\mathbb{Z}_2[x]$ om het uitbreidingsveld $GF(2^2) = GF(4) \cong \mathbb{Z}_2[x]|_{w(x)}$ te construeren.

In [22]:
Z2 = IntegerField(2)

w, xi_powers = Z2.find_primitive_polynomial(2)
print("w(x) =", w)
# xi_powers is formeel nog steeds een veelterm in x
for i in range(len(xi_powers)):
    print("ξ^" + str(i) + " = " + str(xi_powers[i]).replace("x", "ξ"))

w(x) = 1 + x + x²
ξ^0 = 1
ξ^1 = ξ
ξ^2 = 1 + ξ


Hiermee kunnen we dus een uitbreidingsveld $GF(2^2) = GF(4)$ construeren.

In [23]:
GF4 = ExtendedField(Z2, 2, "ξ")
GF4

[0, 1, ξ, 1 + ξ]

We kunnen in dit nieuwe veld opnieuw rekenen

In [24]:
xi = GF4.generator()
for i in range(len(GF4)):
    print(xi ** i)

1
ξ
1 + ξ
1


In [25]:
xi + 3 * GF4[3]

1

In [26]:
xi / GF4.zero()

ZeroDivisionError: Cannot find the exponent for zero. The generator is not a zero divisor.

We kunnen dit veld natuurlijk nog eens uitbreiden tot $GF(16)$

In [27]:
ExtendedField(GF4, 2, "φ")

[0, 1, φ, ξ + φ, ξ + (1 + ξ)φ, 1 + φ, ξ, ξφ, 1 + ξ + ξφ, 1 + ξ + φ, ξ + ξφ, 1 + ξ, (1 + ξ)φ, 1 + (1 + ξ)φ, 1 + ξφ, 1 + ξ + (1 + ξ)φ]

## 1.4 Factorisatie van $x^n - 1$

We vertrekken met de volgende veelterm in $GF(4)[x]$. Mer op dat aagezien $-1 = 1 \text{ (mod 2)}$, deze veelterm ook uitgedrukt kan worden als $x^n + 1$

In [28]:
n = 5
p = Polynomial.x_to_power(5, GF4) - Polynomial.one(GF4)
p

1 + x⁵

We hebben nood aan een uitbreidingsveld $GF(4^k)$ waarin we een element $\beta$ kunnen vinden dat een primitieve $n$-de wortel uit 1 is. $k$ is hierbij het kleinste getal waarvoor $4^k = 1 \text{ (mod } n \text{)}$

In [29]:
beta, k = GF4.nth_root_of_unity(p.degree(), return_k=True)
beta, k

(ξ + (1 + ξ)α, 2)

We zien dat $\beta$ niet in $GF(4)$ gelegen is, maar in $GF(4^k)$. Het is echter wel een oplossing van $p$

In [30]:
beta ** n

1

We bepalen de factorisatie met behulp van de cyclotomische nevenklassen modulo $n$ over $GF(4)$.

In [31]:
cyclotomic_cosets(4, n)

[[0], [1, 4], [2, 3]]

Als we deze machten van $\beta$ combineren, hebben we een factorisatie van $x^n - 1$ over $GF(4)$.

In [32]:
factors = GF4.factor_nth_root(n)
for factor in factors:
    print(factor)

1 + x
1 + (1 + ξ)x + x²
1 + ξx + x²


En inderdaad:

In [33]:
product(factors) == p

True